In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

# Exercises

<hr style="border:2px solid gray">

# Only:

## d. Based on previous year/month/etc., this is up to you.

### - Resample By Month
### - Predict based on previous year

```python
saas_df = pd.read_csv('saas.csv')
saas_df = clean_saas_df(saas_df)
```

<hr style="border:2px solid gray">

# Imports

In [2]:
def clean_saas_df(df):
    # set datetime data type and sort
    df['Month_Invoiced']=pd.to_datetime(df['Month_Invoiced'])
    
    # set 'Date' column as index and sorted
    df = df.set_index("Month_Invoiced").sort_index()
    
    # feature engineering: added 'month' column using index
    df['month'] = df.index.month_name()
    
    # feature engineering: added 'year' column using index
    df['year'] = df.index.year
    
    # change datatypes
    df["year"]  = df["year"].astype('string')
    df["Customer_Id"]  = df["Customer_Id"].astype('string')
    df["Invoice_Id"]  = df["Invoice_Id"].astype('string')
    df["Subscription_Type"]  = df["Subscription_Type"].astype('string')
    
    # rename columns snake_case
    
    df.columns = (df.columns
                .str.replace(' ', '_', regex=True)
                .str.lower()
             )

    return df

In [3]:
def evaluate(target_var):
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 0)
    return rmse

In [4]:
def plot_and_eval(target_var):
    plt.figure(figsize = (12,4))
    plt.plot(train[target_var], label = 'Train', linewidth = 1)
    plt.plot(validate[target_var], label = 'Validate', linewidth = 1)
    plt.plot(yhat_df[target_var])
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.0f}'.format(rmse))
    plt.show()

In [5]:
# Create the empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

# function to store rmse for comparison purposes
def append_eval_df(model_type, target_var):
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var], 'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

<hr style="border:2px solid gray">

# Acquire

In [6]:
saas_df = pd.read_csv('saas.csv')

In [7]:
saas_df = clean_saas_df(saas_df)

In [8]:
# changing 'amount' column to float16
saas_df['amount'] = saas_df['amount'].astype('float16')

## split settings Percentage
```python
train_size = int(len(saas_df) * .5)
validate_size = int(len(saas_df) * .3)
test_size = int(len(saas_df) - train_size - validate_size)
validate_end_index = train_size + validate_size
```

## Percentage split into train, validation, test
```python
train = saas_df[: train_size]
validate = saas_df[train_size : validate_end_index]
test = saas_df[validate_end_index : ]
```

# REDO

# 1. Split data (train/validate/test) and resample by any period except daily. Aggregate using the sum.

In [9]:
train = saas_df[:'2015']
validate = saas_df.loc['2016']
test = saas_df.loc['2017']

In [10]:
# R E S A M P L E
saas_df_resampled_m = train.resample('M').sum()

In [ ]:
### predictions

In [ ]:
# train.diff(365).mean()

In [ ]:
# yhat_df = train.loc['2015'] + train.diff(365).mean()

In [11]:
validate.head(3)

,customer_id,invoice_id,subscription_type,amount,month,year
Month_Invoiced,,,,,,
2016-01-31,1000000,5431894,1.0,5.0,January,2016
2016-01-31,1000003,5431895,0.0,0.0,January,2016
2016-01-31,1000004,5431896,1.0,5.0,January,2016


In [ ]:
# Predict sales using previous cycle method
validate['pred_sales'] = train['2015'].shift(12)

print(df_monthly)

In [ ]:
validate.head(3)

In [ ]:
# Adding Top Line in Validate to Top Line in Predictions DF (yhat)
pd.concat([yhat_df.head(1), validate.head(1)])

# Predict?

In [ ]:
# set yhat_df to index of validate
yhat_df.index = validate.index

len(yhat_df)

In [ ]:
# Plot & Eval target name column in Train DF
for col in train.columns:
    plot_and_eval(target_var = col)
    eval_df = append_eval_df(model_type = 'previous year', target_var = col)

In [ ]:
saas_df.head()

```python
# Predict sales using previous cycle method
saas_df['pred_sales'] = saas_df['sales'].shift(12)

print(df_monthly)
```

<hr style="border:2px solid gray">

# Curriculum

In [ ]:
# monthly delta for year
delta = df.diff()

In [ ]:
# average delta for each year
first_year_delta = delta[delta.index.year == 2014].mean()[0]
second_year_delta = delta[delta.index.year == 2015].mean()[0]

In [ ]:
validate.shape,test.shape

In [ ]:
# Use the average delta to predict the third year
prediction_df = pd.DataFrame(columns=['item_price', 'amount_sold'])
prediction_df.loc[0] = df.loc[df.index.year == 2015].iloc[-1]

In [ ]:
prediction_df

In [ ]:
for i in range(12):
    prediction_df.loc[i+1] = prediction_df.loc[i] + [first_year_delta, second_year_delta]

In [ ]:
# Add the year 2020 to the date index of the prediction DataFrame
prediction_df.index = pd.date_range(start='2020-01-01', periods=12, freq='M')
prediction_df